# Import Deep Learning Probabilities

In [155]:
# Orthophoto
import numpy as np
import pandas as pd

#fortworth_prob_ortho = np.load('D:/Nafiseh/building_type/Orthophoto/Results/probability_fortworth_orthophoto.npy')

#df = pd.read_csv('D:/Nafiseh/building_type/vancouver_prob_test.csv', usecols = lambda x: x not in ['OID_', 'Object_ID_1','Geom','obj_id_lng','object_id_12'])
df = pd.read_csv('D:/Nafiseh/building_type/vancouver_prob_test_2.csv')

df_fortworth = pd.read_csv('D:/Nafiseh/building_type/fortworth_prob_test_2.csv')
#print(fortworth_prob_ortho.shape)


def create_column_names(mode):
    
    dtype1 = ['ortho','lidar']
    
    dtype2 = ['gsv']
    
    NUM_FOLDS = 5

    classes1 = ['industrial','institutional','office_building','other','residential','retail']
    
    classes2 = ['apartment','church','house','industrial','mixed_r_c','office_building','retail']

    prob_columns = []
    
    if mode == 1:

        for d in dtype1:

            for f in range(1,NUM_FOLDS+1):

                for c in classes1:

                    prob_columns.append(f'Prob_{c}_{d}_fold_{f}')

    elif mode == 2:
        
        
        for d in dtype2:

            for f in range(1,NUM_FOLDS+1):

                for c in classes2:

                    prob_columns.append(f'Prob_{c}_{d}_fold_{f}')
                    
    else:
        
       raise Exception('mode should be either 1 or 2') 
        
    return(prob_columns) 

############

vancouver_prob_array_ortho_lidar = np.array(df[create_column_names(1)])

vancouver_prob_array_gsv = np.array(df[create_column_names(2)])

##############

fortworth_prob_array_ortho_lidar = np.array(df_fortworth[create_column_names(1)])

fortworth_prob_array_gsv = np.array(df_fortworth[create_column_names(2)])

###############

print('Vancouver Probability Arrays have shapes: \n', vancouver_prob_array_ortho_lidar.shape, vancouver_prob_array_gsv.shape)

print('Fortworth Probability Arrays have shapes: \n', fortworth_prob_array_ortho_lidar.shape, fortworth_prob_array_gsv.shape)

Vancouver Probability Arrays have shapes: 
 (184, 60) (184, 35)
Fortworth Probability Arrays have shapes: 
 (401, 60) (401, 35)


# Fusion Method: Fuzzy-ranking Based Ensemble Using Gombertz Function

In [156]:
def fuzzy_rank(CF, top):
    R_L = np.zeros(CF.shape)
    for i in range(CF.shape[0]):
        for j in range(CF.shape[1]):
            for k in range(CF.shape[2]):
                R_L[i][j][k] = 1 - math.exp(-math.exp(-2.0*CF[i][j][k]))  #Gompertz Function
    
    K_L = 0.632*np.ones(shape = R_L.shape) #initiate all values as penalty values
    for i in range(R_L.shape[0]):
        for sample in range(R_L.shape[1]):
            for k in range(top):
                a = R_L[i][sample]
                idx = np.where(a==np.partition(a, k)[k])
                #if sample belongs to top 'k' classes, R_L =R_L, else R_L = penalty value
                K_L[i][sample][idx] = R_L[i][sample][idx]

    return K_L

def CFS_func(CF, K_L):
    H = CF.shape[0] #no. of classifiers
    for f in range(CF.shape[0]):
        for i in range(CF.shape[1]):
            idx = np.where(K_L[f][i] == 0.632)
            CF[f][i][idx] = 0
    CFS = 1 - np.sum(CF,axis=0)/H
    return CFS

def Gompertz(CF ,top = 2):

#def Gompertz(top = 2, *argv):
    #L = 0 #Number of classifiers

    #for arg in argv:
        #L += 1

    #num_classes = arg.shape[1]
    #CF = np.zeros(shape = (L,arg.shape[0], arg.shape[1]))

    #for i, arg in enumerate(argv):
        #CF[:][:][i] = arg

    R_L = fuzzy_rank(CF, top) #R_L is with penalties
    
    RS = np.sum(R_L, axis=0)

    CFS = CFS_func(CF, R_L)

    FS = np.multiply(RS,CFS)

    predictions = np.argmin(FS,axis=1)
    return predictions

# Step 1: Fusion of Orthophoto and LiDAR

In [157]:
CF1 = np.copy(vancouver_prob_array_ortho_lidar)

CF1_F = np.copy(fortworth_prob_array_ortho_lidar)

def reshape_confidence_matrix (CF):
    
        
    num_classes = 6
    
    CF_r = np.zeros((int(CF.shape[1]/num_classes), CF.shape[0], num_classes))

    indexes = np.arange(0, CF.shape[1],num_classes)
    
    #print(indexes)
    
    for c in range(num_classes):

        for i in indexes:

            CF_r[int(i/num_classes),:,c] = CF[:,i+c].T

    return(CF_r) 

###########
    
CF_r = reshape_confidence_matrix(CF1)  

#print(CF_r)
#print(CF_r.shape)
predictions_step1 = Gompertz(CF_r)

print(predictions_step1)

#############

CF_r_fortworth = reshape_confidence_matrix(CF1_F)

predictions_step1_fortworth = Gompertz(CF_r_fortworth)

print(predictions_step1_fortworth)

[2 4 2 2 5 5 1 0 4 1 0 0 0 4 2 1 3 4 4 1 0 0 3 4 5 0 0 4 2 3 4 2 5 5 5 0 0
 0 4 0 2 4 2 5 3 4 4 5 1 3 2 1 0 3 0 4 4 4 3 4 4 0 3 0 4 4 4 4 1 1 0 0 2 2
 2 4 3 2 4 4 5 4 1 4 5 0 4 4 0 4 4 0 1 5 4 4 0 4 3 4 1 1 1 2 2 4 4 4 4 4 5
 2 0 4 0 0 0 0 4 4 2 5 3 2 4 4 4 4 5 4 2 2 2 4 4 4 2 4 4 4 5 4 4 4 4 5 4 2
 0 0 1 0 3 4 0 3 5 4 3 2 1 4 1 0 3 1 0 5 0 4 0 4 0 0 0 0 0 0 0 0 0 0 0 0]
[5 1 3 3 3 2 3 3 3 3 2 2 3 3 2 3 3 1 5 2 2 3 1 3 1 3 3 3 3 3 2 2 3 2 2 1 2
 2 3 2 3 3 3 3 1 2 2 2 1 3 3 3 3 2 2 3 3 3 1 2 2 1 2 3 3 3 3 1 1 1 3 3 1 2
 4 2 5 3 5 1 0 5 1 3 3 2 3 3 1 3 2 2 1 1 1 2 3 2 3 3 1 0 3 3 4 3 3 1 1 1 3
 3 2 3 2 3 3 1 3 2 3 5 3 3 1 2 2 3 4 4 1 5 5 3 5 2 2 2 2 3 3 4 4 3 3 4 3 3
 4 3 3 5 2 3 3 2 5 3 4 3 3 3 3 3 3 4 3 2 4 3 3 4 3 4 2 4 2 4 1 2 3 3 3 5 3
 3 2 2 5 3 3 4 3 3 4 4 4 3 4 4 4 3 3 5 5 3 4 4 3 4 4 3 4 3 4 3 3 3 2 4 3 0
 3 4 4 1 3 4 4 4 4 3 2 1 5 3 3 3 3 3 3 3 1 3 3 3 3 2 2 3 3 0 4 4 3 3 3 2 3
 2 3 3 3 3 2 2 2 3 3 3 3 3 3 3 3 3 3 3 4 2 3 3 3 4 4 3 3 2 3 2 3 5 2 4 2 3
 2 3 3 3 3 4 3 2 3 2 3 2 2

# Make GSV Labels Consistent to Step 1 Result (Orthophoto and LiDAR)

In [158]:
# 0 (aprtment) >> 4 (residential)
# 1 (church)  >> 1 (institutional)
# 2 (house)   >> 4 (residential)
# 3 (industrial) >> 0 (industrial)
# 4 (mixed r/c)  >> 5 (retail)
# 5 (office building)  >> 2 (office building)
# 6 (retail)  >>  5 (retail)

def make_gsv_labels_consistent_to_ortho_lidar(prob_array_gsv):
    
    NUM_FOLDS = 5
    
    num_classes_gsv = 7
    
    num_cols = np.shape(prob_array_gsv)[1]
    
    prob_array_gsv_c = np.copy(prob_array_gsv)
    
    for i in range(NUM_FOLDS):

        apt = np.copy(prob_array_gsv[:,0+i*num_classes_gsv])

        house = np.copy(prob_array_gsv[:,2+i*num_classes_gsv])

        industrial = np.copy(prob_array_gsv[:,3+i*num_classes_gsv])

        mixed = np.copy(prob_array_gsv[:,4+i*num_classes_gsv])

        office_b = np.copy(prob_array_gsv[:,5+i*num_classes_gsv])

        retail = np.copy(prob_array_gsv[:,6+i*num_classes_gsv])

        prob_array_gsv_c[:,0+i*num_classes_gsv] = industrial

        prob_array_gsv_c[:,2+i*num_classes_gsv] = office_b

        prob_array_gsv_c[:,3+i*num_classes_gsv] = np.zeros((prob_array_gsv.shape[0],))  # class "other" has zero probability in gsv

        #take the maximum probability between apartment and house

        prob_array_gsv_c[:,4+i*num_classes_gsv] = np.max(np.concatenate((np.expand_dims(apt,axis=1),np.expand_dims(house,axis=1)),axis=1))

        #take the maximum probability between mixed r/c and retail

        prob_array_gsv_c[:,5+i*num_classes_gsv] = np.max(np.concatenate((np.expand_dims(mixed,axis=1), np.expand_dims(retail,axis=1)),axis=1))
        
    # remove redundant columns (you now have 6 classes) 
    
    counter = 0
    
    for j in np.arange(num_classes_gsv-1, num_cols, num_classes_gsv):

        prob_array_gsv_c = np.delete(prob_array_gsv_c,[j-counter], axis=1)
        
        counter += 1
    
    #normalization
    
    prob_array_gsv_c_n = np.copy(prob_array_gsv_c)
    
    for k in range(prob_array_gsv_c.shape[0]):
        
        prob_array_gsv_c_n[k,:] = (prob_array_gsv_c[k,:]-np.min(prob_array_gsv_c[k,:]))/(np.max(prob_array_gsv_c[k,:])-np.min(prob_array_gsv_c[k,:]))
    
    return(prob_array_gsv_c_n) 

###################
        
vancouver_prob_array_gsv_c_n = make_gsv_labels_consistent_to_ortho_lidar(vancouver_prob_array_gsv)   

#print(vancouver_prob_array_gsv_c_n.shape)

####################

fortworth_prob_array_gsv_c_n = make_gsv_labels_consistent_to_ortho_lidar(fortworth_prob_array_gsv)

print(fortworth_prob_array_gsv_c_n.shape)

(401, 30)


In [184]:
########################

CF2 = np.concatenate((vancouver_prob_array_ortho_lidar, vancouver_prob_array_gsv_c_n), axis=1)

#print(CF2.shape)

CF2_r = reshape_confidence_matrix(CF2)

#print(CF2_r)

#print(CF2_r.shape)

predictions = Gompertz(CF2_r)

print(predictions)

##############

CF2_fortworth = np.concatenate((fortworth_prob_array_ortho_lidar, fortworth_prob_array_gsv_c_n), axis=1)

CF2_fortworth_r = reshape_confidence_matrix(CF2_fortworth)

predictions_f = Gompertz(CF2_fortworth_r)

print(predictions_f)
##################

[5 4 4 5 5 5 4 4 4 5 4 4 5 4 4 4 5 4 4 4 5 4 5 4 5 5 4 4 5 4 4 5 5 5 5 4 4
 5 4 4 5 4 5 5 5 4 4 5 4 4 5 4 4 4 5 4 4 4 4 4 4 4 4 4 4 4 4 4 5 4 4 5 4 5
 4 4 4 4 4 4 5 4 4 4 5 4 4 4 4 4 4 5 4 5 4 4 5 4 5 4 4 4 4 5 4 4 4 4 4 4 5
 5 4 4 5 4 4 5 4 4 5 5 5 4 4 4 4 4 5 4 5 4 4 4 4 4 5 4 4 4 5 4 4 4 4 5 4 5
 5 5 4 5 4 4 5 5 5 4 4 5 4 4 5 5 4 5 5 5 5 4 4 4 5 5 5 5 5 5 5 5 5 5 5 5]
[5 5 5 4 5 2 5 5 5 5 4 4 5 4 5 4 5 4 5 5 4 5 5 5 5 5 5 5 5 5 5 5 4 2 5 5 5
 5 5 5 4 5 3 4 4 5 5 5 4 2 5 5 3 5 5 5 5 5 4 4 2 5 2 5 5 4 5 5 4 5 5 5 5 5
 4 5 5 5 5 5 5 5 5 5 5 5 4 5 4 4 4 5 4 5 5 4 3 5 5 5 5 5 5 5 4 5 5 5 5 5 5
 5 5 5 4 4 5 5 5 5 5 5 5 5 5 5 4 4 4 4 5 5 5 4 5 4 4 4 4 5 5 4 4 5 5 4 5 5
 4 5 5 5 5 5 5 5 5 5 4 5 5 5 5 5 5 4 5 5 4 5 5 4 5 4 5 4 5 4 4 5 5 5 5 5 5
 5 5 5 5 5 5 4 5 5 5 4 4 5 4 4 4 5 5 5 5 5 4 4 5 4 5 5 4 5 4 5 5 5 5 4 4 5
 5 4 4 4 5 4 4 5 4 5 5 4 5 5 4 5 5 5 5 5 4 5 5 5 5 5 2 5 5 5 4 5 5 5 5 5 5
 5 5 5 5 4 2 5 5 5 5 5 5 5 5 5 5 4 5 5 4 5 5 5 5 4 4 5 5 5 5 5 5 5 5 4 5 5
 5 5 5 5 5 4 5 5 5 2 5 4 5

# Attach Predictions to the Original CSV File

In [13]:
import pandas as pd

df1 = pd.concat([df, pd.DataFrame(predictions, columns = ['Fuzzy_Fusion_Predictions'])], axis =1)

df1.to_csv('D:/Nafiseh/building_type/vancouver_prob_test_2_new.csv')

#########################################

df2 = pd.concat([df_fortworth, pd.DataFrame(predictions_f, columns = ['Fuzzy_Fusion_Predictions'])], axis =1)

df2.to_csv('D:/Nafiseh/building_type/fortworth_prob_test_2_new.csv')

# Visualize and Save Fusion Maps

## Visualization Function; Adapted to Fortworth Case Study

In [193]:
def visualize_and_save_figure(fusion_array, path):
    
    #orange_patch = mpatches.Patch(color = 'orange', label = 'industrial')
    
    #cyan_patch = mpatches.Patch(color = 'cyan', label='institutional')
    
    darkred_patch = mpatches.Patch(color = 'darkred', label = 'office building')
    
    red_patch = mpatches.Patch(color = 'red', label = 'other')
    
    blue_patch = mpatches.Patch(color = 'blue', label='residential')

    magenta_patch = mpatches.Patch(color = 'magenta', label = 'retail')

    #green_patch = mpatches.Patch(color = 'green', label = 'house')

    yellow_patch = mpatches.Patch(color = 'yellow', label = 'background')

    colors = ['darkred','red','blue','magenta', 'yellow']
    
        
    extent_f = [-97.340242, -97.326509, 32.746281, 32.757833]  # fortworth
    
    ax = plt.gca()
    
    ax.tick_params(labeltop = True, labelright = True)
    
    plt.rcParams['font.family'] = 'Times New Roman'
    
    plt.rcParams['font.size'] = 40
    
    plt.rcParams['figure.figsize'] = [40, 20]
    
    u = np.unique(fusion_array)

    norm = BoundaryNorm(list(np.arange(np.min(u)-1,u[-2]+1,17))+list(np.arange(u[-2]+1,np.max(u)+2,170)), len(u))

    cmap_listed = ListedColormap(colors)

    plt.imshow(fusion_array, cmap = cmap_listed, norm = norm, extent = extent_f)

    plt.legend(handles = [darkred_patch, red_patch, blue_patch, magenta_patch, yellow_patch], loc = 'lower center',bbox_to_anchor = (-0.1,-0.15))

    plt.xlabel('Longitude (deg)', font = 'Times New Roman', fontsize = 50, fontweight = 'bold')
    
    plt.ylabel('Latitude (deg)', font = 'Times New Roman', fontsize = 50, fontweight = 'bold')
    
    plt.xticks(np.arange(-97.340242, -97.326509, 0.005))
    
    plt.yticks(np.arange(32.746281, 32.757833, 0.005))
    
    plt.axis('on')

    plt.savefig(path)

    plt.show()

In [159]:
from PIL import Image

import matplotlib.pyplot as plt

import matplotlib.patches as mpatches

from matplotlib.colors import ListedColormap, BoundaryNorm

################################

# 34 >> office building, 51 >> other, 68 >> residential, 85 >> retail, 255 >> background

################################

gompertz_fortworth = np.array(Image.open('D:/Nafiseh/building_type/fortworth_gompertz_fused.tif'))

#print(np.unique(gompertz_fortworth))

#print(np.unique(gompertz_fortworth), gompertz_fortworth.shape)

#plt.imshow(gompertz_fortworth)

#plt.show()

visualize_and_save_figure(gompertz_fortworth,'D:/Nafiseh/building_type/fortworth_gompertz_fused_figure_2.png')


## Visualization Function; Adapted to Vancouver Case Study

In [207]:
def visualize_and_save_figure_2(fusion_array, path):
    
    #orange_patch = mpatches.Patch(color = 'orange', label = 'industrial')
    
    #cyan_patch = mpatches.Patch(color = 'cyan', label='institutional')
    
    #darkred_patch = mpatches.Patch(color = 'darkred', label = 'office building')
    
    #red_patch = mpatches.Patch(color = 'red', label = 'other')
    
    blue_patch = mpatches.Patch(color = 'blue', label='residential')

    magenta_patch = mpatches.Patch(color = 'magenta', label = 'retail')

    #green_patch = mpatches.Patch(color = 'green', label = 'house')

    yellow_patch = mpatches.Patch(color = 'yellow', label = 'background')

    colors = ['blue','magenta', 'yellow']
    
    extent = [-123.136140, -123.129303, 49.280349, 49.284829]  # vancouver
    
    ax = plt.gca()
    
    ax.get_xaxis().get_major_formatter().set_useOffset(False)
    
    ax.get_yaxis().get_major_formatter().set_useOffset(False)
    
    ax.tick_params(labeltop = True, labelright = True)
    
    plt.rcParams['font.family'] = 'Times New Roman'
    
    plt.rcParams['font.size'] = 40
    
    plt.rcParams['figure.figsize'] = [40,20]
    
    u = np.unique(fusion_array)

    norm = BoundaryNorm(list(np.arange(np.min(u)-1,u[-2]+1,17))+list(np.arange(u[-2]+1,np.max(u)+2,170)), len(u))

    cmap_listed = ListedColormap(colors)

    plt.imshow(fusion_array, cmap = cmap_listed, norm = norm, extent = extent)

    plt.legend(handles = [blue_patch, magenta_patch, yellow_patch], loc = 'lower center',bbox_to_anchor = (-0.01,-0.15))

    plt.xlabel('Longitude (deg)', font = 'Times New Roman', fontsize = 50, fontweight = 'bold')
    
    plt.ylabel('Latitude (deg)', font = 'Times New Roman', fontsize =50, fontweight = 'bold')
    
    plt.xticks(np.arange(-123.136140, -123.129303, 0.003))
    
    plt.yticks(np.arange(49.280349, 49.284829, 0.002))
    
    plt.axis('on')

    plt.savefig(path)

    plt.show()

In [160]:
from PIL.Image import Resampling

import matplotlib

gompertz_vancouver = Image.open('D:/Nafiseh/building_type/vancouver_gompertz_fuse.tif').resize((2560,2560), resample = Resampling.NEAREST)

visualize_and_save_figure_2(gompertz_vancouver, 'D:/Nafiseh/building_type/vancouver_gompertz_fused_figure_2.png')

# Calculate Accuracy Metrics

## Read Ground Truth File

### Pixel-Based Accuracy Assessment

In [161]:
def convert_pil_fusion_map_labels_to_original(fusion_array):
    
    fusion_array [fusion_array == 34] = 2
    
    fusion_array [fusion_array == 51] = 3
    
    fusion_array [fusion_array == 68] = 4
    
    fusion_array [fusion_array == 85] = 5
    
    fusion_array [fusion_array == 255] = 6
    
    return(fusion_array)

In [238]:
from PIL import Image

from PIL.Image import Resampling

import numpy as np

from sklearn.metrics import classification_report

############################## Labels in Ground Truth Map ##############################

# 0 >> background, 20 >> apartment, 38 >> office building, 76 >> garage, 90 >> roof, 
# 105 >> retail, 142 >> industrial, 150 >> house, 179 >> institutional

#########################################################################################

############################### Labels in Fusion Map #####################################

# 34 >> office building, 51 >> other, 68 >> residential, 85 >> retail, 255 >> background

##########################################################################################

################################## Vancouver #############################################

class_names = ['industrial','institutional','office building','other','residential','retail','background']

gt_vancouver = make_gt_labels_consistent_to_fusion_map(np.array(Image.open(r'F:\Backup_Nafiseh_Hard_Drives\BIC_GSV.tar\BIC_GSV\vancouver_test_region\vancouver_groundtruth_gray.png')))

gompertz_vancouver = convert_pil_fusion_map_labels_to_original(np.array(Image.open('D:/Nafiseh/building_type/vancouver_gompertz_fuse.tif').resize((2560,2560), resample = Resampling.NEAREST)))

#print(np.unique(gt_vancouver), np.unique(gompertz_vancouver))

#print(gt_vancouver.shape, gompertz_vancouver.shape)

print(classification_report(np.ravel(gt_vancouver), np.ravel(gompertz_vancouver), target_names = class_names))

################################# Fortworth ##################################################

class_names = ['industrial','institutional','office building','other','residential','retail','background']

gt_fortworth = make_gt_labels_consistent_to_fusion_map(np.array(Image.open(r'F:\Backup_Nafiseh_Hard_Drives\BIC_GSV.tar\BIC_GSV\fortworth_test_region\fortworth_groundtruth_gray.png')))

gompertz_fortworth = convert_pil_fusion_map_labels_to_original(np.array(Image.open('D:/Nafiseh/building_type/fortworth_gompertz_fused.tif').resize((5120,5120), resample = Resampling.NEAREST)))

#print(np.unique(gt_fortworth), np.unique(gompertz_fortworth))

#print(gt_fortworth.shape, gompertz_fortworth.shape)

print(classification_report(np.ravel(gt_fortworth), np.ravel(gompertz_fortworth), target_names = class_names))

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf

                 precision    recall  f1-score   support

     industrial       0.00      0.00      0.00     43668
  institutional       0.00      0.00      0.00     15902
office building       0.00      0.00      0.00    180637
          other       0.00      0.00      0.00    115192
    residential       0.45      0.41      0.43   1214447
         retail       0.16      0.42      0.24    382736
     background       0.87      0.85      0.86   4601018

       accuracy                           0.70   6553600
      macro avg       0.21      0.24      0.22   6553600
   weighted avg       0.70      0.70      0.70   6553600



C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf

                 precision    recall  f1-score   support

     industrial       0.00      0.00      0.00    444049
  institutional       0.00      0.00      0.00    166926
office building       0.52      0.03      0.06   4421130
          other       0.80      0.14      0.24    315595
    residential       0.07      0.31      0.12    495108
         retail       0.03      0.49      0.05    214216
     background       0.89      0.87      0.88  20157376

       accuracy                           0.68  26214400
      macro avg       0.33      0.26      0.19  26214400
   weighted avg       0.78      0.68      0.69  26214400



### Object-Based Accuracy Assessment

In [162]:
###########################################
#from osgeo import gdal

#fortworth_gt_gdal = gdal.Open(r'F:\Backup_Nafiseh_Hard_Drives\BIC_GSV.tar\BIC_GSV\fortworth_test_region\fortworth_groundtruth_gray.png')

#fortworth_gt = fortworth_gt_gdal.ReadAsArray()

#print(np.unique(fortworth_gt))

import pandas as pd

df_pred_fortworth = pd.read_csv('D:/Nafiseh/building_type/fortworth_prob_test_2_new.csv')

df_gt_fortworth = pd.read_csv('D:/Nafiseh/building_type/fortworth_gt_zst.csv')

df_m_f = pd.merge(df_pred_fortworth, df_gt_fortworth, on= 'CFWLAND_ID')

gt_f = np.array(df_m_f.GT_2)

predictions_f_ = np.array(df_m_f.Fuzzy_Fusion_Predictions)

#df_m_f
##################################

df_pred_vancouver = pd.read_csv('D:/Nafiseh/building_type/vancouver_prob_test_2_new.csv')

df_gt_vancouver = pd.read_csv('D:/Nafiseh/building_type/vancouver_gt_zst_4.csv')

df_m_v = pd.merge(df_pred_vancouver, df_gt_vancouver, on='obj_id_lng')

gt_v = np.array(df_m_v.GT_2)

predictions_v_ = np.array(df_m_v.Fuzzy_Fusion_Predictions)

#df_m_v

## Make Ground Truth Labels Consistent to Fusion Map

In [163]:
def make_gt_labels_consistent_to_fusion_map(gt):

    # labels in fusion map:
    
    # 0 >> industrial, 1 >> institutional, 2 >> office building, 3 >> other, 4 >> residential, 5 >> retail

    gt[gt == 0] = 6

    gt[(gt == 29) | (gt == 150)] = 4

    gt[gt == 38] = 2

    gt[gt == 142] = 0

    gt[gt == 105] = 5

    gt[gt == 179] = 1

    gt[(gt == 76) | (gt == 90)] = 3
    
    return(gt)


#############################3
    
fortworth_gt_new_label = np.array(make_gt_labels_consistent_to_fusion_map(gt_f))

print(np.unique(fortworth_gt_new_label))

print(fortworth_gt_new_label.shape)

###########################

vancouver_gt_new_label = np.array(make_gt_labels_consistent_to_fusion_map(gt_v))

print(np.unique(vancouver_gt_new_label))

print(vancouver_gt_new_label.shape)

[1 2 3 4 5 6]
(401,)
[2 3 4 5 6]
(184,)


In [164]:
from sklearn.metrics import classification_report

from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay

class_names = ['institutional','office building','other','residential','retial','background']

print(classification_report(fortworth_gt_new_label, predictions_f_, target_names = class_names))



disp = ConfusionMatrixDisplay(confusion_matrix(fortworth_gt_new_label, predictions_f_), display_labels = class_names)

disp.plot()

plt.show()

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf

                 precision    recall  f1-score   support

  institutional       0.00      0.00      0.00        14
office building       0.33      0.03      0.06        96
          other       1.00      0.03      0.05       108
    residential       0.08      0.11      0.09        75
         retial       0.05      0.43      0.09        35
     background       0.00      0.00      0.00        73

       accuracy                           0.07       401
      macro avg       0.24      0.10      0.05       401
   weighted avg       0.37      0.07      0.05       401



In [165]:
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

class_names_2 = ['office building','other','residential','retail','background']

print(classification_report(vancouver_gt_new_label, predictions_v_, target_names = class_names_2))

disp = ConfusionMatrixDisplay(confusion_matrix(vancouver_gt_new_label, predictions_v_), display_labels = class_names_2)

disp.plot()

plt.show()

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf

                 precision    recall  f1-score   support

office building       0.00      0.00      0.00         2
          other       0.00      0.00      0.00        21
    residential       0.39      0.63      0.48        70
         retail       0.35      0.35      0.35        71
     background       0.00      0.00      0.00        20

       accuracy                           0.38       184
      macro avg       0.15      0.20      0.17       184
   weighted avg       0.28      0.38      0.32       184



# Calculate Confusion Matrix

## Fortworth

In [19]:
def make_labels_same_as_gt(array):
    
    array [array == 4] = 29
    
    array [array == 5] = 105
    
    array[array == 6] = 0

    array[array == 4] = 29

    array[array == 2] = 38

    array[array == 3] = 90

    array[array == 5] = 105

    array[array == 0] = 142

    array[array == 4] = 150

    array[array == 3] = 179
    
    return(array)

In [20]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

df = pd.read_csv('D:/Nafiseh/building_type/fortworth_fuzzy_spj_new_table.csv')

df_fuzzy = pd.read_csv('D:/Nafiseh/building_type/fortworth_prob_test_2_new.csv')

class_names = ['background','apartment','office building','roof','garage','retail','industrial','house','church']

df_final = pd.merge(df,df_fuzzy, on = 'ADDRESS')

plt.rcParams['font.size'] = 10

disp = ConfusionMatrixDisplay(confusion_matrix(np.array(df_final['gridcode']), make_labels_same_as_gt(np.array(df_final['Fuzzy_Fusion_Predictions']))), display_labels = class_names)

disp.plot()
                                          
#_,ax=plt.subplots(figsize=(25,15))

#sns.set (font_scale = 2)                                          

#sns.heatmap(confusion_matrix/np.sum(confusion_matrix,axis=1)[:,None], annot=True, fmt='.2%', cmap='Blues',ax=ax)

#sns.heatmap(confusion_matrix, annot=True, fmt='.2%', cmap='Blues',ax=ax)

#ax.set_xlabel('Predicted labels',fontweight='bold',fontsize = 20)
#ax.set_ylabel('True labels',fontweight='bold',fontsize = 20)
#ax.set_title('Confusion Matrix',fontweight='bold',fontsize = 26) 

    
#ax.xaxis.set_ticklabels(['office building','other','residential','retail'], fontsize = 18)
                           

#ax.yaxis.set_ticklabels(['office building','other','residential','retail'], fontsize = 18)
                        

#plt.savefig('D:/Nafiseh/building_type/Building_Type_Manuscript/confusion_matrix_vancouver_fuzzy.tif')

plt.show()

## Vancouver

In [173]:
def make_labels_same_as_gt(array):
    
    array [array == 4] = 29
    
    array [array == 5] = 105
    
    return(array)

In [176]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

df = pd.read_csv('D:/Nafiseh/building_type/vancouver_prob_test_spj_new.csv')

class_names = ['background','apartment','office building','garage','retail','industrial','house','church']

disp = ConfusionMatrixDisplay(confusion_matrix(np.array(df['gridcode']), make_labels_same_as_gt(np.array(df['Fuzzy_Fusion']))), display_labels = class_names)

disp.plot()
                                          
#_,ax=plt.subplots(figsize=(25,15))

#sns.set (font_scale = 2)                                          

#sns.heatmap(confusion_matrix/np.sum(confusion_matrix,axis=1)[:,None], annot=True, fmt='.2%', cmap='Blues',ax=ax)

#sns.heatmap(confusion_matrix, annot=True, fmt='.2%', cmap='Blues',ax=ax)

#ax.set_xlabel('Predicted labels',fontweight='bold',fontsize = 20)
#ax.set_ylabel('True labels',fontweight='bold',fontsize = 20)
#ax.set_title('Confusion Matrix',fontweight='bold',fontsize = 26) 

    
#ax.xaxis.set_ticklabels(['office building','other','residential','retail'], fontsize = 18)
                           

#ax.yaxis.set_ticklabels(['office building','other','residential','retail'], fontsize = 18)
                        

#plt.savefig('D:/Nafiseh/building_type/Building_Type_Manuscript/confusion_matrix_vancouver_fuzzy.tif')

plt.show()